In [ ]:
#importing necessary libraries
import geopy.geocoders
import pandas as pd
import numpy as np

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Reading the dataset
df = pd.read_csv("/content/drive/MyDrive/Project_work/covid_19_clean_complete.csv")

In [ ]:
df.tail()

,Province/State,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
49063,NaN,Sao Tome and Principe,0.186400,6.613100,7/27/2020,865,14,734,117,Africa
49064,NaN,Yemen,15.552727,48.516388,7/27/2020,1691,483,833,375,Eastern Mediterranean
49065,NaN,Comoros,-11.645500,43.333300,7/27/2020,354,7,328,19,Africa
49066,NaN,Tajikistan,38.861000,71.276100,7/27/2020,7235,60,6028,1147,Europe
49067,NaN,Lesotho,-29.610000,28.233600,7/27/2020,505,12,128,365,Africa


In [ ]:
#Only considering death counts more than 1000
df = df[df.Deaths >= 1000]

In [ ]:
aggregation_functions = {'Deaths': 'sum', 'Country': 'first'}
aggregate_df = df.groupby(df['Country']).aggregate(aggregation_functions)

In [ ]:
aggregate_df

,Deaths,Country
Country,,
Afghanistan,18542,Afghanistan
Algeria,18378,Algeria
Argentina,67380,Argentina
Bangladesh,95327,Bangladesh
Belgium,959706,Belgium
Bolivia,54756,Bolivia
Brazil,3932185,Brazil
Canada,643129,Canada
Chile,304377,Chile


In [ ]:
#Imporing necessary libraries for the map visualization
#Nominatim is a tool to search OpenStreetMap data by address or location (geocoding). Nominatim is included in the GeoPy in the GeoPy Python library.
#Folium is a powerful Python library that helps you create several types of Leaflet maps.
#Marker clusters When the map is zoomed out nearby markers are combined together into a cluster, which is separated out when the map zoom level is closer. In this post I show you how Folium marker clusters are easy to set up and use.
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster

In [ ]:
geolocator = Nominatim(user_agent='world_map')
def geolocate(Country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(Country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

In [ ]:
aggregate_df['coord'] = aggregate_df['Country'].apply(lambda x : geolocate(x))

In [ ]:
aggregate_df['coord']

Country
Afghanistan                           (33.7680065, 66.2385139)
Algeria                                (28.0000272, 2.9999825)
Argentina                           (-34.9964963, -64.9672817)
Bangladesh                            (24.4769288, 90.2934413)
Belgium                                (50.6402809, 4.6667145)
Bolivia                             (-17.0568696, -64.9912286)
Brazil                                    (-10.3333333, -53.2)
Canada                               (61.0666922, -107.991707)
Chile                               (-31.7613365, -71.3187697)
China                                  (35.000074, 104.999927)
Colombia                               (4.099917, -72.9088133)
Dominican Republic                   (19.0974031, -70.3028026)
Ecuador                              (-1.3397668, -79.3666965)
Egypt                                 (26.2540493, 29.2675469)
France                                  (46.603354, 1.8883335)
Germany                               (51.08341

In [ ]:
aggregate_df['coord'].reset_index(drop=True)

0                     (33.7680065, 66.2385139)
1                      (28.0000272, 2.9999825)
2                   (-34.9964963, -64.9672817)
3                     (24.4769288, 90.2934413)
4                      (50.6402809, 4.6667145)
5                   (-17.0568696, -64.9912286)
6                         (-10.3333333, -53.2)
7                    (61.0666922, -107.991707)
8                   (-31.7613365, -71.3187697)
9                      (35.000074, 104.999927)
10                     (4.099917, -72.9088133)
11                   (19.0974031, -70.3028026)
12                   (-1.3397668, -79.3666965)
13                    (26.2540493, 29.2675469)
14                      (46.603354, 1.8883335)
15                    (51.0834196, 10.4234469)
16    (15.687100699999998, -90.12265453178017)
17                   (15.2572432, -86.0755145)
18                    (22.3511148, 78.6677428)
19                   (-2.4833826, 117.8902853)
20                    (32.6475314, 54.5643516)
21           

In [ ]:
#using a for loop for the latitudes and append the coordinates with it
latitude=[]
for y in [0]:
    for x in range(0,44):
        latitude.append(aggregate_df['coord'][x][y])
aggregate_df['latitude'] = latitude

In [ ]:
#using a for loop for longitude and append the coordinates with it
longitude = []
for y in [1]:
    for x in range(0,44):
        longitude.append(aggregate_df['coord'][x][y])
aggregate_df['longitude'] = longitude

In [ ]:
covid19_deaths_world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(covid19_deaths_world_map)

In [ ]:
for i in range(len(aggregate_df)):
        lat = aggregate_df.iloc[i]['latitude']
        long = aggregate_df.iloc[i]['longitude']
        radius=5
        popup_text = """
                    {} dead patients <br>"""
        popup_text = popup_text.format('{:.2f}'.format(aggregate_df.iloc[i]['Deaths']))
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)

In [ ]:
covid19_deaths_world_map